In [2]:
import pandas as pd

# 사용자별 슈퍼 명단 만들기

In [ ]:
# 단독 사용 예시: Azure 또는 Upstage에서 텍스트만 추출

# Azure만 사용
# from modules.core.extractors.azure_extractor import get_azure_extractor
# azure = get_azure_extractor(enable_cache=False)
# text = azure.extract_from_image(Path("debug2/.../page_2_original_image.png"))
# print(text)

# Upstage만 사용
# from modules.core.extractors.upstage_extractor import get_upstage_extractor
# upstage = get_upstage_extractor(enable_cache=False)
# text = upstage.extract_from_image(Path("debug2/.../page_2_original_image.png"))
# print(text)

# bbox 포함 raw 결과가 필요하면 (하이라이트 UI 등)
# raw = azure.extract_from_image_raw(image_path=path)  # 또는 image_bytes=bytes
# raw = upstage.extract_from_image_raw(image_path=path)
# raw["pages"][0]["words"]  # 각 단어의 text, boundingBox

# 사용자별 슈퍼 명단 만들기

In [ ]:
# 최근 1년치 판매처-슈퍼 매핑
master_code = pd.read_excel("/Users/nongshim/Downloads/条件マスターコード一覧26.01.29.xlsx")
print(master_code.shape)
master_code.columns=["판매처코드", "판매처명", "대표슈퍼코드", "대표슈퍼명", "담당자ID", "담당자명"]
print(len(set(master_code["담당자ID"])))
master_code.head()

(425, 6)
14


,판매처코드,판매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명
0,1303017,株式会社日本アクセス広域リテール営業本部九州支店鹿児島営業所,6035526,株式会社南九州ファミリーマート,N3000039,八塚琴音
1,1302976,株式会社トーカン商品統括部（CVS),6003788,ファミリーマート,N3000034,佐瀬雄一
2,1303568,株式会社日本アクセス広域リテール営業本部加工食品飲料部,6003788,ファミリーマート,N3000034,佐瀬雄一
3,1303567,株式会社日本アクセス広域リテール営業本部東北営業課,6003788,ファミリーマート,N3000034,佐瀬雄一
4,1303569,株式会社日本アクセス広域リテール営業本部中部・北陸営業課,6003788,ファミリーマート,N3000034,佐瀬雄一


In [ ]:
# 슈퍼 담당자 매핑
df = pd.read_excel("/Users/nongshim/Downloads/代表スーパーコード一覧.XLSX")
df = df.drop_duplicates()
print(df.shape)
print(len(set(df["INTRANET ID"])))
df.columns = ['대표슈퍼코드', '대표슈퍼명', '담당자ID', '담당자명', 'ID']
df.head() 
# o30240021

(1445, 5)
16


,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID
0,6000015,■有限会社木村屋,N3000041,澤野亮,o30250003
1,6000104,■株式会社千葉薬品,N3000011,佐々木俊文,o30240005
2,6000131,■株式会社京成ストア,N3000041,澤野亮,o30250003
3,6000181,■株式会社くすりの福太郎,N3000011,佐々木俊文,o30240005
4,6000185,■Ａコープ,N3000011,佐々木俊文,o30240005


In [126]:
df[df["담당자ID"].isin(["N3000025", "N3000013"])]

,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID
453,6020609,■株式会社日立物流,N3000013,営業管理 営業社員,-
577,6033130,■その他,N3000013,営業管理 営業社員,-
768,6056563,■株式会社JR東日本リテールネット,N3000025,◆ＣＶＳ,-
1430,6196420,■辛メーター株式会社,N3000013,営業管理 営業社員,-


In [127]:
master_code[master_code["담당자ID"].isin(["N3000013"])]

,판매처코드,판매처명,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명
10,1300206,株式会社マルイチ産商食品事業部長野支店,6033130,あいにー食品館,N3000013,その他
20,1300206,株式会社マルイチ産商食品事業部長野支店,6033130,株式会社田村商店,N3000013,その他
30,1300142,株式会社トーカン,6033130,清水屋,N3000013,その他
47,1300002,株式会社日本アクセス北関東支店,6033130,カインズ,N3000013,その他
66,1304968,ヤマエ久野株式会社東京支店食品課,6033130,Firsto,N3000013,その他
79,1301657,三井食品株式会社業務用食材本部卸営業部関信越営業所,6033130,エーコープ他 一般店,N3000013,その他
80,1301282,ボーキ佐藤株式会社秋田支店,6033130,株式会社英雄,N3000013,その他
105,1300071,ボーキ佐藤株式会社新潟支店,6033130,スポット,N3000013,その他
106,1300176,丸大堀内株式会社秋田支店,6033130,その他,N3000013,その他
125,1300068,ボーキ佐藤株式会社福島営業所,6033130,船山商店 卸町支店,N3000013,その他


In [132]:
col = ['대표슈퍼코드', '대표슈퍼명', '담당자ID', '담당자명']
super_user_map = pd.concat([df[col].drop_duplicates(), master_code[col].drop_duplicates()]).sort_values(by='담당자ID',ignore_index=True)
super_user_map["대표슈퍼명"] = super_user_map["대표슈퍼명"].apply(lambda x: x.replace("■", ""))
super_user_map["대표슈퍼명"] = super_user_map["대표슈퍼명"].apply(lambda x: x.replace("株式会社", ""))
super_user_map = pd.merge(super_user_map, df[["담당자ID","ID"]].drop_duplicates(), on='담당자ID', how='left')
super_user_map

,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID
0,6021019,信濃屋食品,N3000003,長田悟,o30240015
1,6049544,有限会社八百賢,N3000003,長田悟,o30240015
2,6049800,新印食品,N3000003,長田悟,o30240015
3,6035512,与野フードセンター,N3000003,長田悟,o30240015
4,6164500,マエダ(マークスグループ),N3000003,長田悟,o30240015
...,...,...,...,...,...
1827,6000870,スーパーキタムラ,N3000041,澤野亮,o30250003
1828,6154863,(株)新茨城食品商会,N3000041,澤野亮,o30250003
1829,6003706,ウェルパーク,N3000041,澤野亮,o30250003
1830,6160904,有限会社ナカムラフーズ,N3000041,澤野亮,o30250003


In [137]:
super_user_map[super_user_map["ID"]=="o30240021"]

,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID
1635,6020451,有限会社アズーロ,N3000039,八塚琴音,o30240021
1636,6051380,大賀食品,N3000039,八塚琴音,o30240021
1637,6160916,SOMES,N3000039,八塚琴音,o30240021
1638,6019534,(株)九州シジシー,N3000039,八塚琴音,o30240021
1639,6146470,戸村精肉本店,N3000039,八塚琴音,o30240021
...,...,...,...,...,...
1741,6035504,タイヨー（九州）,N3000039,八塚琴音,o30240021
1742,6047677,下田商事,N3000039,八塚琴音,o30240021
1743,6152386,カーニバル,N3000039,八塚琴音,o30240021
1744,6014315,スーパーまるまつ,N3000039,八塚琴音,o30240021


In [133]:
super_user_map.to_csv("database/super_import.csv", index=False)

# 단가 명단 만들기

In [ ]:
unit_price = pd.read_excel("/Users/nongshim/Downloads/★단가리스트★26.02.10.xls", sheet_name="価格表2306", skiprows=2)
unit_price = unit_price[["商品コード", "商品名", "内容量\n換算値", "問屋　　仕切", "本部長　　ｃｔ", "JANコード　      　　　    (ﾊﾗﾙ→UPC)"]]
unit_price.columns = ["제품코드", "제품명", "제품용량", "시키리", "본부장", "JANCD"]
unit_price.head()

,제품코드,제품명,제품용량,시키리,본부장,JANCD
0,101003042,辛カップラーメン,68.0,140.0,88.0,8801043030694
1,101003583,辛ラーメンキムチ カップ,68.0,140.0,88.0,8801043038096
2,101004421,辛ラーメン激辛カップ,68.0,140.0,88.0,8801043052429
3,101002078,本場韓国コムタンラーメン,75.0,140.0,88.0,8801043011549
4,101005016,ノグリラーメンカップ(HOT),62.0,140.0,88.0,8801043059602


In [174]:
unit_price.to_csv("database/unit_price.csv", index=False)

# 유사도 매핑

In [ ]:
# 슈퍼명 유사도 테스트 (db_manager와 동일한 개념, CSV에서 테스트)
# DB는 pg_trgm similarity() 사용. 여기서는 difflib으로 유사도 0~1 계산
import pandas as pd
from pathlib import Path
from difflib import SequenceMatcher
import re

def split_name_and_capacity(name):
    """
    "제품명" 등에서 용량(예: １２０Ｇ, 120g, 90ML 등)를 분리해서 (이름, 용량) 튜플로 반환.
    용량이 없으면 (원본, None) 반환.
    단, 단위가 그람(g/ｇ/G/Ｇ)일 때는 숫자만 추출해서 반환.
    """
    # 전각/반각/소문자 등 다양한 표기(숫자+단위) (g, G, ｇ, ML 등)
    # 아래 패턴은 마지막에 등장하는 용량(예: １２０Ｇ, 120g, 90ML, 12コ) 등 추출
    # 'Ｇ'도 포함하도록 단위 집합 확장, 전각 및 반각 대소문자 모두 포함!
    pattern = re.compile(
        r"([０-９0-9]+\.?[０-９0-9]*)([gGｇＧmMｍＭlLリットルﾘｯﾄﾙ個コ袋])\s*$"
    )
    # 전각 숫자를 반각으로 바꾸기 위한 테이블
    to_hankaku = str.maketrans("０１２３４５６７８９", "0123456789")
    name = name.strip()
    m = pattern.search(name)
    if m:
        num = m.group(1).translate(to_hankaku)
        unit = m.group(2)
        # 그람 단위(g, G, ｇ, Ｇ)이면 숫자만 남기고, 아니면 단위 붙여 반환
        if unit in ["g", "G", "ｇ", "Ｇ"]:
            cap = num
        else:
            cap = f"{num}{unit}"
        base = name[: m.start()].strip()
        return base, cap
    return name, None


def similarity(a: str, b: str) -> float:
    """두 문자열 유사도 0~1 (pg_trgm과 비슷한 개념)."""
    a, b = (a or "").strip(), (b or "").strip()
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a, b).ratio()

def find_similar_supers(
    query: str,
    csv_path: str,
    col: str,
    top_k: int = 10,
    min_similarity: float = 0.0,
    sub_col: str = None,
    sub_query: str = None,
    sub_min_similarity: float = 0.0,
):
    """
    입력한 슈퍼명과 super_import.csv의 대표슈퍼명(또는 지정 컬럼) 중 유사도 높은 순으로 반환.
    2개 이상의 컬럼 유사도 필터 지원: col과 sub_col을 각각 비교.
    """
    path = Path(csv_path)
    if not path.exists():
        raise FileNotFoundError(f"파일 없음: {path}")
    df = pd.read_csv(path, encoding="utf-8")
    if col not in df.columns:
        raise KeyError(f"컬럼 없음: {col}. 사용 가능: {list(df.columns)}")
    if sub_col and sub_col not in df.columns:
        raise KeyError(f"서브 컬럼 없음: {sub_col}. 사용 가능: {list(df.columns)}")

    query = (query or "").strip()
    sub_query = (sub_query or "").strip() if sub_query is not None else None

    rows = []
    for _, row in df.iterrows():
        # 1차 컬럼(주 컬럼) 유사도 계산
        name = (row.get(col) or "")
        if isinstance(name, float) and pd.isna(name):
            name = ""
        name = str(name).strip()
        primary_score = similarity(query, name)
        # 2차 컬럼(옵션) 유사도 계산
        if sub_col and sub_query is not None:
            sub_value = (row.get(sub_col) or "")
            if isinstance(sub_value, float) and pd.isna(sub_value):
                sub_value = ""
            sub_value = str(sub_value).strip()
            sub_score = similarity(sub_query, sub_value)
        else:
            sub_score = None

        # 1차 필터링 (주 컬럼)
        if primary_score >= min_similarity:
            # 2차 필터링 (서브 컬럼, 없으면 패스)
            if (sub_col and sub_query is not None):
                if sub_score >= sub_min_similarity:
                    row_dict = {**row.to_dict(), f"{col}_similarity": round(primary_score, 4), f"{sub_col}_similarity": round(sub_score, 4)}
                    rows.append(row_dict)
            else:
                row_dict = {**row.to_dict(), "similarity": round(primary_score, 4)}
                rows.append(row_dict)
    # 정렬 기준: 1차는 주 컬럼, 둘 다 있으면 (주, 보조 동시 정렬)
    if sub_col and sub_query is not None:
        rows.sort(key=lambda x: (x[f"{col}_similarity"], x[f"{sub_col}_similarity"]), reverse=True)
    else:
        rows.sort(key=lambda x: x.get("similarity", 0), reverse=True)
    return pd.DataFrame(rows).head(top_k)

In [185]:
INPUT_SUPER_NAME = "岩田産業(株)     本社"   # 예: オザム, バザール, 信濃屋, マエダ, ファミリーマート 등
top = find_similar_supers(INPUT_SUPER_NAME, csv_path="database/super_import.csv", col="대표슈퍼명", top_k=5, min_similarity=0.2)
print(f"입력: '{INPUT_SUPER_NAME}' → 유사도 상위 {len(top)}건")
display(top)

입력: '岩田産業(株)     本社' → 유사도 상위 5건


,대표슈퍼코드,대표슈퍼명,담당자ID,담당자명,ID,similarity
0,6052877,岩田産業,N3000039,八塚琴音,o30240021,0.4444
1,6033130,(株)Ooki,N3000013,その他,-,0.2857
2,6154032,(株)上野商事,N3000032,三澤暢太,o30240006,0.2857
3,6033130,尾家産業東京支社,N3000013,その他,-,0.2727
4,6033130,尾家産業札幌支社,N3000013,その他,-,0.2727


In [190]:
INPUT_SUPER_NAME = "農心ジャパン 辛カップラーメン ６８Ｇ"
base_name, capacity = split_name_and_capacity(INPUT_SUPER_NAME)
print(f"이름: '{base_name}', 용량: '{capacity}'")

top = find_similar_supers(base_name, csv_path="database/unit_price.csv", col="제품명",top_k=5, min_similarity=0.2, sub_col="제품용량", sub_query=capacity)
print(f"입력: '{base_name}' → 유사도 상위 {len(top)}건")
display(top)

이름: '農心ジャパン 辛カップラーメン', 용량: '68'
입력: '農心ジャパン 辛カップラーメン' → 유사도 상위 5건


,제품코드,제품명,제품용량,시키리,본부장,JANCD,제품명_similarity,제품용량_similarity
0,101003042,辛カップラーメン,68.0,140.0,88.0,8801043030694,0.6957,0.6667
1,101003068,★ハラル辛カップラーメン,68.0,148.0,108.0,★031146270606,0.5926,0.6667
2,101002007,ユッケジャンカップ,86.0,91.0,78.0,8801043010139,0.5000,0.3333
3,101001960,旨激辛ラーメン,65.0,91.0,78.0,4582214631165,0.4545,0.3333
4,101000491,辛ラーメン3P,360.0,378.0,225.0,8801043157506,0.4545,0.2857


In [187]:
INPUT_SUPER_NAME = "農心 辛ラーメン 袋 １２０Ｇ"
base_name, capacity = split_name_and_capacity(INPUT_SUPER_NAME)
print(f"이름: '{base_name}', 용량: '{capacity}'")

top = find_similar_supers(base_name, csv_path="database/unit_price.csv", col="제품명",top_k=5, min_similarity=0.2, sub_col="제품용량", sub_query=capacity)
print(f"입력: '{base_name}' → 유사도 상위 {len(top)}건")
display(top)

이름: '農心 辛ラーメン 袋', 용량: '120'
입력: '農心 辛ラーメン 袋' → 유사도 상위 5건


,제품코드,제품명,제품용량,시키리,본부장,JANCD,제품명_similarity,제품용량_similarity
0,101000461,辛ラーメン1P,120.0,126.0,78.0,8801043150606,0.5882,0.7500
1,101001960,旨激辛ラーメン,65.0,91.0,78.0,4582214631165,0.5882,0.2857
2,101000491,辛ラーメン3P,360.0,378.0,225.0,8801043157506,0.5882,0.2500
3,101001977,辛ラーメン5P,600.0,630.0,345.0,8801043009720,0.5882,0.2500
4,101003042,辛カップラーメン,68.0,140.0,88.0,8801043030694,0.5556,0.2857
